# Pregunta 1, parte 1

## Importacion

In [ ]:
import pandas as pd
from paths import *
from functools import reduce

In [ ]:
# Documentación del módulo 'paths'.
# import paths
# help(paths)

## Carga de datos

### Creación de la tabla mezclada

In [ ]:
# Cargar todos los csv en dos listas distinguidas por 
# la aparicion de 'furnished' en el nombre

# Lista de DataFrames (Sin 'furnished' en el nombre)
L_DF_a = [pd.read_csv(dict_csv_mc_a[wNN]) for wNN in L_WNN]
# Lista de DataFrames (Con 'furnished' en el nombre)
L_DF_f = [pd.read_csv(dict_csv_mc_f[wNN]) for wNN in L_WNN]

In [ ]:
# se unen todos los data_frame en cada lista con el comando concat
DF_a = pd.concat(L_DF_a)
DF_f = pd.concat(L_DF_f)

# Se crea una columna con tantos ceros como la cantidad de filas de DF_a
# y con tantos unos como la cantidad de filas de DF_f 
furnished_col = pd.DataFrame(data = {'furnished':[0]*(DF_a.shape[0]) + [1]*(DF_f.shape[0])})

# Concateno los data frames DF_a y DF_f y le agrego al final la columna furnished_col
df = pd.concat([DF_a,DF_f], ignore_index=True) 
df = pd.concat([df,furnished_col], axis=1) 

In [ ]:
# Quitamos las filas que sean iguales en todas las columnas con excepcion de
# la columna furnished y reindexamos
df.drop_duplicates(subset = df.columns[:-1], ignore_index=True, inplace=True)

# comentario: si hay filas duplicadas con distintos valores de furnished, se elimina la primera

In [ ]:
# reportar si Reporte si existen observaciones de archivos con texto ’furnished’ que no est´en
# contenidos en archivos con texto ’all’

## Pregunta 1, parte 2

### Limpieza de datos

In [ ]:
# Creamos valores a reemplazar
valores_nulos = {
    'property_type|rent_type|location':'Sin nombre',
    'price':'$0',
    'n_rooms':'NA',
    'n_bath':'NA',
    'surface':'0.0 m2',
    'details':'',
    'url':'',
    'metrocuadrado_index':0.0,
    'furnished':0
}

In [ ]:
# Reemplazamos los valores
df.fillna(value=valores_nulos, inplace=True)
df

In [ ]:
# prueba